In [13]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import os
import cv2

batch_size = 64

# Load MNIST dataset
(train_images, train_labels), (test_images_mnist, test_labels_mnist) = tf.keras.datasets.mnist.load_data()

# Convert images and labels to tensors
train_images = tf.convert_to_tensor(train_images, dtype=tf.float32)
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)

test_folder = 'D:\mlr\digit_detection\digit_images'
test_images = []
test_labels = [5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1, 1]

# Iterate through .png files
for filename in os.listdir(test_folder):
    if filename.endswith(".png"):
        # Read the image and convert it to grayscale
        image_path = os.path.join(test_folder, filename)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (28, 28))  # Resize to desired size
        image = image.astype(np.float32) / 255.0 
        # Append the image and label to the test datasets
        test_images.append(image)

# Convert the lists to NumPy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Convert images and labels to tensors
test_images = tf.convert_to_tensor(test_images, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)

# Combine MNIST testing dataset with custom testing dataset
test_images_combined = tf.concat([test_images_mnist, test_images], axis=0)
test_labels_combined = tf.concat([test_labels_mnist, test_labels], axis=0)

# Create test dataset
test_dataset_combined = tf.data.Dataset.from_tensor_slices((test_images_combined, test_labels_combined))
test_dataset_batched = test_dataset_combined.batch(batch_size)

# Create a TensorFlow dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

# Shuffle and batch the training dataset
train_dataset = train_dataset.shuffle(buffer_size=len(train_images)).batch(batch_size)

print(test_images_combined.shape)
print(test_labels_combined.shape)
# Choose CPU or GPU to execute the code
devices = tf.config.list_physical_devices("GPU")
if len(devices) > 0:
    device = "GPU"
else:
    device = "CPU"

print(f"Using {device} device")

# Neural network model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu', input_shape= (28,28,1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='softmax'))
model.add(tf.keras.layers.Dense(10))
model.summary()

# Choose the optimizer algorithm, loss function, and evaluation metrics
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.00005),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

# Training process
num_epochs = 5
model.fit(
    train_dataset,
    epochs=num_epochs,
    batch_size=batch_size,
)

# Evaluate the model on the combined test dataset
test_loss, test_accuracy = model.evaluate(test_dataset_batched)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

print("Done!")


(10025, 28, 28)
(10025,)
Using CPU device
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 3, 64)          36928     
                                                                 
 flatten_1 (

In [18]:
# Assuming i is the index of the image you want to predict
i = 10002

# Get the i-th image from the test dataset
image = test_images_combined[i]
image = tf.expand_dims(image, axis=0)  # Add a batch dimension

# Perform prediction
predictions = model.predict(image)

# Get the predicted label
predicted_label = tf.argmax(predictions[0]).numpy()

# Print the predicted label
print(f"Predicted Label: {predicted_label}")

1/1 [==============================] - 0s 21ms/step
Predicted Label: 1
